# Movie Review Analysis Outline
## import necessary python libraries


## Import the Data
## Clean up Data (extract data we need (i.e. stars, and perhaps text of review)

In [20]:
# output.txt is created by running a python script to convert the format of the movie data
# into a format that spark readily converts into a RDD

# "transformData.py" is this script which is intended to be run once on our data


# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky
movies_txt = sc.textFile("output.txt").map(lambda x: (x.split('/')))

print (movies_txt.collect())

#the next step
# parse and take care of funky symbols in summary and text <- normalize to root words maybe?
def prepData (datapath):
    pass


[[u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text'], [u'B003AI2VGA', u'A141HP4LYPWMSR', u'Brian E. Erland "Rainbow Sphinx"', u'7', u'7', u'3.0', u'1182729600', u'"There Is So Much Darkness Now ~ Come For The Miracle"', u'Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br ', u'><br ', u'>Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the "Wounds of Christ" (stigmata) appear in her palms. She also claims to have 

## Divide into 2 datasets (one for training and one for testing) 

## Train classifier

In [ ]:
# train the classifier 



## Run classifier on testing set